<img src="https://raw.githubusercontent.com/rbizoi/PythonFormation/main/images/e-brasil.png" width="850">

# Les imports et initialisations des variables

In [52]:
from datetime import datetime
import pandas as pd, numpy as np, os, warnings, seaborn as sns, pickle, re, unicodedata
from datetime import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font1 = fm.FontProperties(size=20)
font2 = fm.FontProperties(size=24)

%matplotlib inline

if int(str(sns.__version__).split('.')[1]) > 8 : 
    plt.style.use('seaborn-v0_8-darkgrid')
else:
    plt.style.use('seaborn-darkgrid')
    
sns.set(font_scale=2)
warnings.filterwarnings(action="ignore")

## Liste fichiers de données 

In [53]:
!dir ..\donnees\ebrasil

 Le volume dans le lecteur F s'appelle Data
 Le num‚ro de s‚rie du volume est D4F7-8E12

 R‚pertoire de F:\PythonFormation\donnees\ebrasil

14/02/2024  09:47    <DIR>          .
14/02/2024  09:47    <DIR>          ..
06/10/2019  20:27         9ÿ033ÿ957 olist_customers_dataset.csv
06/10/2019  20:27        61ÿ273ÿ883 olist_geolocation_dataset.csv
06/10/2019  20:27        17ÿ654ÿ914 olist_orders_dataset.csv
06/10/2019  20:27        15ÿ438ÿ671 olist_order_items_dataset.csv
06/10/2019  20:27         5ÿ777ÿ138 olist_order_payments_dataset.csv
06/10/2019  20:27        14ÿ409ÿ007 olist_order_reviews_dataset.csv
06/10/2019  20:27         2ÿ379ÿ446 olist_products_dataset.csv
28/09/2020  07:52           177ÿ799 olist_sellers_dataset.csv
17/01/2024  08:49             2ÿ542 product_category_name_translation.csv
               9 fichier(s)      126ÿ147ÿ357 octets
               2 R‚p(s)  86ÿ159ÿ405ÿ056 octets libres


## Changement de répertoire

In [54]:
os.chdir(r"../donnees")

# DataFrame $items$

In [55]:
donnees = pd.read_csv(os.path.join('ebrasil', 'olist_order_items_dataset.csv'))
donnees = donnees.merge(pd.read_parquet('./ecommerce/orders.parquet')[['order_id', 'purchase_timestamp']],on='order_id')

In [56]:
donnees.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,purchase_timestamp
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,2017-09-13 08:59:02
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,2017-04-26 10:53:06
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,2018-01-14 14:33:31
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,2018-08-08 10:00:35
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,2017-02-04 13:57:51


In [57]:
donnees.dtypes

order_id                       object
order_item_id                   int64
product_id                     object
seller_id                      object
shipping_limit_date            object
price                         float64
freight_value                 float64
purchase_timestamp     datetime64[ns]
dtype: object

## Conversions des dates

In [58]:
donnees['shipping_limit']= pd.to_datetime(donnees.shipping_limit_date, format='%Y-%m-%d %H:%M:%S',errors='ignore')
donnees['limit']         = (donnees.shipping_limit - donnees.purchase_timestamp).dt.seconds/60/60

In [59]:
donnees.drop(columns=['shipping_limit_date','purchase_timestamp'],inplace=True)

In [60]:
donnees.dtypes

order_id                  object
order_item_id              int64
product_id                object
seller_id                 object
price                    float64
freight_value            float64
shipping_limit    datetime64[ns]
limit                    float64
dtype: object

In [61]:
donnees.isna().sum()

order_id          0
order_item_id     0
product_id        0
seller_id         0
price             0
freight_value     0
shipping_limit    0
limit             0
dtype: int64

In [62]:
donnees.order_id.nunique(),donnees.shape

(98666, (112650, 8))

In [63]:
donnees.groupby(['order_id','product_id']).agg('count').shape

(102425, 6)

In [64]:
donnees.groupby(['order_id','product_id','seller_id']).agg('count').shape

(102425, 5)

In [65]:
donnees.groupby(['order_id','order_item_id','product_id','seller_id']).agg('count').shape,donnees.shape

((112650, 4), (112650, 8))

In [66]:
donnees.head()

,order_id,order_item_id,product_id,seller_id,price,freight_value,shipping_limit,limit
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29,2017-09-19 09:45:35,0.775833
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93,2017-05-03 11:05:13,0.201944
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87,2018-01-18 14:48:30,0.249722
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79,2018-08-15 10:10:18,0.161944
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14,2017-02-13 13:57:51,0.000000


## Sauvegarde en parquet

In [67]:
donnees.to_parquet('./ecommerce/items.parquet',compression='gzip', engine='pyarrow')

In [68]:
pd.read_parquet('./ecommerce/items.parquet').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   order_id        112650 non-null  object        
 1   order_item_id   112650 non-null  int64         
 2   product_id      112650 non-null  object        
 3   seller_id       112650 non-null  object        
 4   price           112650 non-null  float64       
 5   freight_value   112650 non-null  float64       
 6   shipping_limit  112650 non-null  datetime64[ns]
 7   limit           112650 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(3)
memory usage: 6.9+ MB


In [69]:
!dir ecommerce

 Le volume dans le lecteur F s'appelle Data
 Le num‚ro de s‚rie du volume est D4F7-8E12

 R‚pertoire de F:\PythonFormation\donnees\ecommerce

14/02/2024  16:20    <DIR>          .
14/02/2024  16:20    <DIR>          ..
14/02/2024  15:45         4ÿ525ÿ883 customers.parquet
14/02/2024  15:48         1ÿ122ÿ602 geolocation.parquet
14/02/2024  16:35         4ÿ726ÿ507 items.parquet
14/02/2024  16:08         4ÿ829ÿ801 items_products.parquet
14/02/2024  15:45        27ÿ443ÿ040 orders.csv
14/02/2024  15:45         9ÿ269ÿ678 orders.parquet
14/02/2024  15:46         9ÿ725ÿ278 orders_payments.parquet
14/02/2024  15:47        11ÿ010ÿ093 orders_payments_reviews_p.parquet
14/02/2024  16:20        22ÿ022ÿ482 orders_pric.parquet
14/02/2024  15:46         2ÿ375ÿ117 payments.parquet
14/02/2024  16:08           963ÿ000 products.parquet
14/02/2024  15:47         7ÿ858ÿ269 reviews.parquet
14/02/2024  15:47         3ÿ201ÿ554 reviews_p.parquet
14/02/2024  15:49            91ÿ808 sellers.parquet
              

# DataFrame $products$ 

In [70]:
donnees = pd.read_csv(os.path.join('ebrasil', 'olist_products_dataset.csv'))
donnees.rename(columns={col:col.replace('product_','') for col in donnees.columns[1:]},inplace=True)

In [71]:
donnees.head()

,product_id,category_name,name_lenght,description_lenght,photos_qty,weight_g,length_cm,height_cm,width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [72]:
categories = pd.read_csv(os.path.join('ebrasil', 'product_category_name_translation.csv'))
categories.rename(columns={col:col.replace('product_','') for col in categories.columns},inplace=True)
categories.head()

,category_name,category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [73]:
donnees = donnees.merge(categories,how='left',on='category_name')

In [74]:
donnees[donnees.category_name_english.isna()][['category_name',
                                               'category_name_english']].drop_duplicates()

,category_name,category_name_english
105,NaN,NaN
1628,pc_gamer,NaN
5821,portateis_cozinha_e_preparadores_de_alimentos,NaN


In [75]:
donnees.loc[donnees.category_name_english.isna() & 
                    (donnees.category_name == 'pc_gamer'),'category_name_english'] = 'pc_gamer'
donnees.category_name_english[donnees.category_name_english.isna() & 
                    (donnees.category_name == 'portateis_cozinha_e_preparadores_de_alimentos') ] = 'kitchenware_tools_and_gadget'

In [76]:
donnees.category_name_english.fillna('not documented',inplace=True)

In [77]:
donnees.drop(columns='category_name',inplace=True)
donnees.rename(columns={'category_name_english':'category_name'},inplace=True)

In [78]:
donnees.head()

,product_id,name_lenght,description_lenght,photos_qty,weight_g,length_cm,height_cm,width_cm,category_name
0,1e9e8ef04dbcff4541ed26657ea517e5,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,art
2,96bd76ec8810374ed1b65e291975717f,46.0,250.0,1.0,154.0,18.0,9.0,15.0,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,27.0,261.0,1.0,371.0,26.0,4.0,26.0,baby
4,9dc1a7de274444849c219cff195d0b71,37.0,402.0,4.0,625.0,20.0,17.0,13.0,housewares


In [27]:
donnees.isna().sum()

product_id              0
name_lenght           610
description_lenght    610
photos_qty            610
weight_g                2
length_cm               2
height_cm               2
width_cm                2
category_name           0
dtype: int64

In [28]:
donnees.to_parquet('./ecommerce/products.parquet',compression='gzip', engine='pyarrow')

In [29]:
donnees.shape

(32951, 9)

In [30]:
items = pd.read_parquet('./ecommerce/items.parquet')

In [31]:
items.product_id.nunique(),donnees.shape              

(32951, (32951, 9))

In [32]:
items = items.merge(donnees,on='product_id')

In [33]:
items.head()

,order_id,order_item_id,product_id,seller_id,price,freight_value,shipping_limit,limit,name_lenght,description_lenght,photos_qty,weight_g,length_cm,height_cm,width_cm,category_name
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.9,13.29,2017-09-19 09:45:35,0.775833,58.0,598.0,4.0,650.0,28.0,9.0,14.0,cool_stuff
1,130898c0987d1801452a8ed92a670612,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,55.9,17.96,2017-07-05 02:44:11,14.864167,58.0,598.0,4.0,650.0,28.0,9.0,14.0,cool_stuff
2,532ed5e14e24ae1f0d735b91524b98b9,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,64.9,18.33,2018-05-23 10:56:25,0.508889,58.0,598.0,4.0,650.0,28.0,9.0,14.0,cool_stuff
3,6f8c31653edb8c83e1a739408b5ff750,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.9,16.17,2017-08-07 18:55:08,0.273889,58.0,598.0,4.0,650.0,28.0,9.0,14.0,cool_stuff
4,7d19f4ef4d04461989632411b7e588b9,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.9,13.29,2017-08-16 22:05:11,0.275278,58.0,598.0,4.0,650.0,28.0,9.0,14.0,cool_stuff


# DataFrame $items - products$ 

## Sauvegarde en parquet

In [34]:
items.to_parquet('./ecommerce/items_products.parquet',compression='gzip', engine='pyarrow')

In [35]:
pd.read_parquet('./ecommerce/items_products.parquet').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   order_id            112650 non-null  object        
 1   order_item_id       112650 non-null  int64         
 2   product_id          112650 non-null  object        
 3   seller_id           112650 non-null  object        
 4   price               112650 non-null  float64       
 5   freight_value       112650 non-null  float64       
 6   shipping_limit      112650 non-null  datetime64[ns]
 7   limit               112650 non-null  float64       
 8   name_lenght         111047 non-null  float64       
 9   description_lenght  111047 non-null  float64       
 10  photos_qty          111047 non-null  float64       
 11  weight_g            112632 non-null  float64       
 12  length_cm           112632 non-null  float64       
 13  height_cm           112632 no

In [36]:
!dir ecommerce

 Le volume dans le lecteur F s'appelle Data
 Le num‚ro de s‚rie du volume est D4F7-8E12

 R‚pertoire de F:\PythonFormation\donnees\ecommerce

14/02/2024  15:37    <DIR>          .
14/02/2024  15:37    <DIR>          ..
14/02/2024  15:45         4ÿ525ÿ883 customers.parquet
14/02/2024  15:48         1ÿ122ÿ602 geolocation.parquet
14/02/2024  16:08         4ÿ726ÿ507 items.parquet
14/02/2024  16:08         4ÿ829ÿ801 items_products.parquet
14/02/2024  15:45        27ÿ443ÿ040 orders.csv
14/02/2024  15:45         9ÿ269ÿ678 orders.parquet
14/02/2024  15:46         9ÿ725ÿ278 orders_payments.parquet
14/02/2024  15:47        11ÿ010ÿ093 orders_payments_reviews_p.parquet
14/02/2024  15:46         2ÿ375ÿ117 payments.parquet
14/02/2024  16:08           963ÿ000 products.parquet
14/02/2024  15:47         7ÿ858ÿ269 reviews.parquet
14/02/2024  15:47         3ÿ201ÿ554 reviews_p.parquet
14/02/2024  15:49            91ÿ808 sellers.parquet
              13 fichier(s)       87ÿ142ÿ630 octets
               2 R